# Visualize fluorescence preprocessing pipeline for whole recordings

This notebook is a step-by-step visualization of the preprocessing pipeline with output plots and explanations. The scripts `preprocess_data_SC.py` and `preprocess_data_DC.py` automate this workflow for all experiments.

In [16]:
%matplotlib widget

import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import savgol_filter

from ipywidgets import interact, interactive, fixed
import ipywidgets as widgets

import paths
import functions_preprocessing as fpp
import functions_io as f_io
import functions_plotting as fp
from functions_utils import find_zone_and_behavior_episodes, add_episode_data

f_io.check_dir_exists(paths.figure_directory)

## Load the data

For convenience, load the already preprocesed data. This dataset contains the columns 'gcamp_raw' and 'auto_raw', which are the raw data

In [5]:
mouse = 1
day = 3

data_df = f_io.load_preprocessed_data(mouse, day)

## Visualize raw traces

Here we notice some disconinuities and jumps in the data set. We need to figure out what to do with them to clean this up.

In [96]:
def plot_two_traces(time, trace1, trace2, **kwargs):    
    fig, axes = plt.subplots(nrows=2, sharex=True, figsize=(9,6), **kwargs)
    
    axes[0] = fp.plot_fluorescence_min_sec(time, trace1, ax=axes[0])
    axes[0].set_ylabel('Fluorescence (AU)')
    
    axes[1] = fp.plot_fluorescence_min_sec(time, trace2, ax=axes[1])
    axes[1].set_ylabel('Fluorescence (AU)')
    axes[1].set_xlabel('Time')
    
    return fig, axes

In [9]:
fig, axes = plot_two_traces(data_df.time, data_df.gcamp_raw, data_df.auto_raw)
axes[0].set_title('Raw GCaMP')
axes[1].set_title('Raw Auto')
plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Remove NaNs

These traces have some NaNs in them. This affects later processing. The function `fpp.remove_nans` finds these values and removes them by interpolating.

In [10]:
gcamp = data_df.gcamp.to_numpy()
gcamp = fpp.remove_nans(gcamp)

auto = data_df.auto.to_numpy()
auto = fpp.remove_nans(auto)

In [11]:
fig, axes = plot_two_traces(data_df.time, gcamp, auto)
axes[0].set_title('NaN-Removed GCaMP')
axes[1].set_title('NaN-Removed Auto')
plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Find periods where the signal is lost

Notice that sometimes the signal is lost, and a flat line is found in the data. We identify these sections by calculating the derivative of both channels. Where the derivative is zero, the signal is flat and presumably lost. Find where this occurs in both signals, and replace with np.NaN

In [12]:
d_gcamp = np.r_[0, np.abs(np.diff(gcamp))]
d_auto = np.r_[0, np.abs(np.diff(auto))]
gcamp_d0 = np.where(d_gcamp == 0)[0]
auto_d0 = np.where(d_auto == 0)[0]
shared_zero = np.unique(np.concatenate((gcamp_d0, auto_d0)))

In [11]:
# fig, axes = plt.subplots(nrows=2, ncols=2, sharex=True)
# axes[0][0].plot(data_df.time, gcamp)
# axes[1][0].plot(data_df.time, d_gcamp)
# axes[0][1].plot(data_df.time, auto, c='orange')
# axes[1][1].plot(data_df.time, d_auto, c='orange')

In [13]:
# fig, axes = plt.subplots(nrows=2, ncols=1, sharex=True)
# axes[0].plot(data_df.time, gcamp)
# axes[1].plot(data_df.time, auto, c='orange')

# High pass filter to remove the decay

Notice that there is a decay throughout the experiment. High pass filter (>0.1 Hz) to remove this.

In [64]:
import scipy.signal as sig
fs = 40 


def butter_highpass(cutoff, order, fs):
    nyq = 0.5 * fs
    high_cut = cutoff / nyq
    b, a = sig.butter(order, high_cut, btype='highpass')
    return b, a

In [77]:
def plot_hpf(signal, cutoff, order, fs):
    
    b, a = butter_highpass(cutoff, order, fs)
    signal_hpf = sig.filtfilt(b, a, signal)
    
    fig, axes = plot_two_traces(data_df.time, signal, signal_hpf)
    axes[0].set_title('Original Signal')
    axes[1].set_title('High-Pass Filtered Signal')
    plt.tight_layout()
    
    return signal_hpf

In [80]:
y=interactive(plot_hpf,
              signal=fixed(gcamp), 
              cutoff=(0.01, 1.0, 0.05),
              order=(1, 10, 1), 
              fs=fixed(fs))
display(y)
gcamp_hpf = y.result

interactive(children=(FloatSlider(value=0.46, description='cutoff', max=1.0, min=0.01, step=0.05), IntSlider(v…

In [109]:
y=interactive(plot_hpf,
              signal=fixed(auto), 
              cutoff=(0.01, 1.0, 0.05),
              order=(1, 10, 1), 
              fs=fixed(fs))
display(y)
auto_hpf = y.result

interactive(children=(FloatSlider(value=0.46, description='cutoff', max=1.0, min=0.01, step=0.05), IntSlider(v…

# Low pass filter to remove noise artifacts

Notice that there is some jitter still in the plot. Low pass filter the data with a frequency that makes sense for the Ca2+ indicator. If GCaMP6s, this is ~10 Hz.

In [70]:
def butter_lowpass(cutoff, order, fs):
    nyq = 0.5 * fs
    low_cut = cutoff / nyq
    b, a = sig.butter(order, low_cut, btype='lowpass')
    return b, a

In [85]:
def plot_lpf(signal, cutoff, order, fs):
    
    b, a = butter_lowpass(cutoff, order, fs)
    signal_lpf = sig.filtfilt(b, a, signal)
    
    fig, axes = plot_two_traces(data_df.time, signal, signal_lpf)
    axes[0].set_title('High-Pass Filtered  Signal')
    axes[1].set_title('Low-Pass Filtered Signal')
    plt.tight_layout()
    
    return signal_lpf

In [90]:
y=interactive(plot_lpf,
              signal=fixed(gcamp_hpf), 
              cutoff=(10, (fs/2)-1, 0.5),
              order=(1, 10, 1), 
              fs=fixed(fs))
display(y)
gcamp_lpf = y.result

interactive(children=(FloatSlider(value=14.5, description='cutoff', max=19.0, min=10.0, step=0.5), IntSlider(v…

In [108]:
y=interactive(plot_lpf,
              signal=fixed(auto_hpf), 
              cutoff=(10, (fs/2)-1, 0.5),
              order=(1, 10, 1), 
              fs=fixed(fs))
auto_lpf = y.result

NameError: name 'auto_hpf' is not defined

# Apply SavGol filter for smoothing

In [104]:
def plot_savgol(signal, kernel=5, poly_order=2):
    if poly_order > kernel:
        poly_order = kernel-1
        
    signal_savgol = savgol_filter(signal, kernel, poly_order)

    fig, axes = plot_two_traces(data_df.time, signal, signal_savgol, sharey=True)
    axes[0].set_title('Filtered  Signal')
    axes[1].set_title('Smoothed Signal')
    plt.tight_layout()
    
    return signal_savgol

In [106]:
y=interactive(plot_savgol,
              signal=fixed(gcamp_lpf), 
              kernel=(3, 39, 2),
              poly_order=(1, 5, 1))

display(y)
gcamp_smooth = y.result

interactive(children=(IntSlider(value=5, description='kernel', max=39, min=3, step=2), IntSlider(value=2, desc…

In [107]:
y=interactive(plot_savgol,
              signal=fixed(auto_lpf), 
              kernel=(3, 39, 2),
              poly_order=(1, 5, 1))

display(y)
auto_smooth = y.result

NameError: name 'auto_lpf' is not defined

# Remove periods where signal is lost

This has to be applied after filtering, because the filters can't handle NaNs

In [33]:
gcamp_smooth[shared_zero] = np.NaN
auto_smooth[shared_zero] = np.NaN

In [35]:
fig, axes = plt.subplots(nrows=2, ncols=1, sharex=True)
axes[0].plot(data_df.time, gcamp_lps)
axes[1].plot(data_df.time, auto_lps, c='orange')

# Remove movement artifacts

Let's take the derivative of the traces and see what's going on with the jumps.

In [18]:
d_gcamp = np.r_[0, np.abs(np.diff(gcamp))]
d_auto = np.r_[0, np.abs(np.diff(auto))]

In [21]:
mean_d_gcamp = np.mean(d_gcamp)
mean_d_auto = np.mean(d_auto)
std_d_gcamp = np.std(d_gcamp)
std_d_auto = np.std(d_auto)

In [25]:
fig, axes = plt.subplots(nrows=2, ncols=2, sharex=True)
axes[0][0].plot(data_df.time, gcamp)
axes[1][0].axhline(mean_d_gcamp)
axes[1][0].axhline(2*std_d_gcamp, color='r')
axes[1][0].plot(data_df.time, d_gcamp)
axes[0][1].plot(data_df.time, auto, c='orange')
axes[1][1].plot(data_df.time, d_auto, c='orange')
axes[1][1].axhline(mean_d_auto)
axes[1][1].axhline(2*std_d_auto, color='r')